<a href="https://colab.research.google.com/github/devshaik444/RandomNumber_generation/blob/master/SSQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
np.random.seed(0)
import operator

class FIFO:
  def __init__(self):
    #self.interarrivals= [0.4,1.2,0.5,1.7,0.2,1.6,0.2,1.4,1.9,.7]  #exponential distribution mean 1/3
    #self.service_times= [2.2,0.5,0.3,1.1,3.5,0.6,0.9,1.5,1.2,.9]
    self.interarrivals= list(np.random.exponential(1/3.0,15))
    self.service_times=list(np.random.exponential(1/4.0,15))
    print(self.interarrivals)
    print(self.service_times)
    self.clock = 0.0

    self.next_arrival = self.interarrivals.pop(0)
    self.next_departure1 = float('inf')
    self.next_departure2 = float('inf')

    self.num_in_queue = 0
    self.times_of_arrivalqueue = []  # store times of arrivals who are waiting in the queue
    self.service_times_in_queue = []  # store service times of waiting customers in the queue

    self.total_delay = 0.0
    self.num_of_delays = 0.0
    self.area_under_q = 0.0
    self.area_under_b1 = 0.0
    self.area_under_b2 = 0.0
    self.number_of_customers_in_queue = 0.0 #total customer count korbey jara queue tey w8 korey per iteration

    self.server1_status = 0
    self.server2_status = 0  # 0 for IDLE , 1 for BUSY
    self.last_event_time = 0.0  # we will need to store last event clock time##total simulation tym

  def start(self):
    while self.num_of_delays <= 10:
      self.timing()

  def timing(self):
        
    self.clock = min(self.next_arrival, self.next_departure1, self.next_departure2)  # First set clock to minimum time of next event
    self.update_register()##q(t) ,b(t) ar jnno

    if self.next_arrival < self.next_departure1 and self.next_arrival < self.next_departure2:
      print("Arrival at Clock:" + str(self.clock))
      self.arrival()

    elif self.next_departure1 < self.next_departure2:
      print("Departure of server 1 at " + str(self.clock))
      self.departure1()

    elif self.next_departure2 < self.next_departure1:
      print("Departure of server 2 at " + str(self.clock))
      self.departure2()


    print("Server Status 1:" + str(self.server1_status))
    print("Server Status 2:" + str(self.server2_status))
    print("Times of arrivals in Queue: " + str(self.times_of_arrivalqueue))
    print("Service times in Queue: " + str(self.service_times_in_queue))
    print("Number of Delays: " + str(self.num_of_delays))
    print("Total Delay:" + str(self.total_delay))
    print("Next Arrival Time: " + str(self.next_arrival))
    print("Next Departure Time of s1: " + str(self.next_departure1))
    print("Next Departure Time of s2: " + str(self.next_departure2))
    print("area under q(t):" + str(self.area_under_q))
    print("area under b1(t):" + str(self.area_under_b1))
    print("area under b2(t):" + str(self.area_under_b2))
    print("last event time"+ str(self.last_event_time))
    print("Expected average Delay:" + str(self.total_delay / 6))
    print("UTILIZATION of server1:" + str(self.area_under_b1/self.last_event_time))
    print("UTILIZATION of server2:" + str(self.area_under_b2/self.last_event_time))
    print("Average number of customers in queue:" + str(self.area_under_q / self.last_event_time))
    print(" ")

  def arrival(self):
            # Schedule next arrival , new_arrival = previous_arrival + inter_arrival time of next customer
    self.next_arrival += self.interarrivals.pop(0)

    if self.server1_status == 0:  # server is idle
      self.server1_status = 1  # make server BUSY
      delay = 0.0  # so delay is zero
      self.total_delay += delay
      self.num_of_delays += 1  # increase the number of customers delayed

      self.next_departure1 = self.clock + self.service_times.pop(0)

    elif self.server2_status == 0:  # server is idle
      self.server2_status = 1  # make server BUSY
      delay = 0.0  # so delay is zero
      self.total_delay += delay
      self.num_of_delays += 1  # increase the number of customers delayed

                # schedule next departure, pop the first element of service_times list to get service time of this customer
      self.next_departure2 = self.clock + self.service_times.pop(0)

    else:  # Server is BUSY
                # increase queue length, this customer will have to wait in the queue
      self.num_in_queue += 1

                # store the arrival time and service time of this customer in seperate lists
      self.times_of_arrivalqueue.append(self.clock)
      self.service_times_in_queue.append(self.service_times.pop(0))

  def departure1(self):
            # check number of customers in the queue
    if self.num_in_queue == 0:  # if no customer in the queue
                # make server IDLE
      self.server1_status = 0
                # schedule next departure= infinity
      self.next_departure1 = float('infinity')

    else:
                # if queue not empty, pop one customer, decrease queue length
      self.num_in_queue -= 1
      self.num_of_delays += 1
                # AS FIFO, pop first arrival and service time from the queue. IF LIFO we have to pop last arrival and service time
                # For SJF, finf the index of minimum service time from  service_times_in_queue list.
                # Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.

      arrival = self.times_of_arrivalqueue.pop(0)

      delay = self.clock - arrival
      self.total_delay += delay
      self.next_departure1 = self.clock + self.service_times_in_queue.pop(0)

  def departure2(self):
            # check number of customers in the queue
    if self.num_in_queue == 0:  # if no customer in the queue
      self.server2_status = 0        
      self.next_departure2 = float('infinity')

    else:
                # if queue not empty, pop one customer, decrease queue length
      self.num_in_queue -= 1
      self.num_of_delays += 1
                # AS FIFO, pop first arrival and service time from the queue. IF LIFO we have to pop last arrival and service time
                # For SJF, finf the index of minimum service time from  service_times_in_queue list.
                # Then pop the arrival of that index from times_of_arrivalqueue for delay count and others.

      arrival = self.times_of_arrivalqueue.pop(0)

      delay = self.clock - arrival
      self.total_delay += delay
      self.next_departure2 = self.clock + self.service_times_in_queue.pop(0)



  def update_register(self):
    time_differnce = self.clock - self.last_event_time
    self.area_under_q += self.num_in_queue * time_differnce
    self.area_under_b1 += self.server1_status * time_differnce
    self.area_under_b2 += self.server2_status * time_differnce
    self.last_event_time = self.clock##


class LIFO:
  def __init__(self):
    #self.interarrivals= [0.4,1.2,0.5,1.7,0.2,1.6,0.2,1.4,1.9,.7,1.6,1.8]  #exponential distribution mean 1/3
    #self.service_times= [2.2,0.5,0.3,1.1,3.5,0.6,0.9,1.5,1.2,0.9,0.2,0.6]
    self.interarrivals= list(np.random.exponential(1/3.0, 15))
    self.service_times=list(np.random.exponential(1/4.0,15))
    print(self.interarrivals)
    print(self.service_times)
    self.clock = 0.0

    self.next_arrival = self.interarrivals.pop(0)
    self.next_departure1 = float('inf')
    self.next_departure2 = float('inf')

    self.num_in_queue = 0
    self.times_of_arrivalqueue = []  # store times of arrivals who are waiting in the queue
    self.service_times_in_queue = []
    
    self.total_delay = 0.0
    self.num_of_delays = 0.0
    self.area_under_q = 0.0
    self.area_under_b1 = 0.0
    self.area_under_b2 = 0.0
    
    self.server1_status = 0
    self.server2_status = 0  # 0 for IDLE , 1 for BUSY
    self.last_event_time = 0.0  # we will need to store last event clock time

        
  def start(self):
    while self.num_of_delays <= 10:
      self.timing()

  def timing(self):
    
    self.clock = min(self.next_arrival, self.next_departure1, self.next_departure2)  # First set clock to minimum time of next event
    self.update_register()

    if self.next_arrival < self.next_departure1 and self.next_arrival < self.next_departure2:
      print("Arrival at Clock:" + str(self.clock))
      self.arrival()

    elif self.next_departure1 < self.next_departure2:
      print("Departure of server 1 at " + str(self.clock))
      self.departure1()
      

    elif self.next_departure2 < self.next_departure1:
      print("Departure of server 2 at " + str(self.clock))
      self.departure2()


    print("Server Status 1:" + str(self.server1_status))
    print("Server Status 2:" + str(self.server2_status))
    print("Times of arrivals in Queue: " + str(self.times_of_arrivalqueue))
    print("Service times in Queue: " + str(self.service_times_in_queue))
    print("Number of Delays: " + str(self.num_of_delays))
    print("Total Delay:" + str(self.total_delay))
    print("Next Arrival Time: " + str(self.next_arrival))
    print("Next Departure Time of s1: " + str(self.next_departure1))
    print("Next Departure Time of s2: " + str(self.next_departure2))
    print("area under q(t):" + str(self.area_under_q))
    print("area under b1(t):" + str(self.area_under_b1))
    print("area under b2(t):" + str(self.area_under_b2))
    print("last event time"+ str(self.last_event_time))
    print("UTILIZATION of server1:" + str(self.area_under_b1/self.last_event_time))
    print("UTILIZATION of server2:" + str(self.area_under_b2/self.last_event_time))
    print("Expected average Delay:" + str(self.total_delay / 6))
    print("Average number of customers in queue:" + str(self.area_under_q / self.last_event_time))
    print("last event time"+ str(self.last_event_time))
    print(" ")


  def arrival(self):
    self.next_arrival += self.interarrivals.pop(0)

    if self.server1_status == 0:  # server is idle
      self.server1_status = 1  # make server BUSY
      delay = 0.0  # so delay is zero
      self.total_delay += delay
      self.num_of_delays += 1  # increase the number of customers delayed

      self.next_departure1 = self.clock + self.service_times.pop(0)

    elif self.server2_status == 0:  # server is idle
      self.server2_status = 1  # make server BUSY
      delay = 0.0  # so delay is zero
      self.total_delay += delay
      self.num_of_delays += 1  # increase the number of customers delayed

                # schedule next departure, pop the first element of service_times list to get service time of this customer
      self.next_departure2 = self.clock + self.service_times.pop(0)

    else:  # Server is BUSY
                # increase queue length, this customer will have to wait in the queue
      self.num_in_queue += 1

                # store the arrival time and service time of this customer in seperate lists
      self.times_of_arrivalqueue.append(self.clock)
      self.service_times_in_queue.append(self.service_times.pop(0))
       
       
  def departure1(self):
    if self.num_in_queue == 0:  # if no customer in the queue
      self.server1_status = 0
      self.next_departure1 = float('infinity')

    else:
      self.num_in_queue -= 1
      self.num_of_delays += 1

      arrival = self.times_of_arrivalqueue.pop(- 1)

      delay = self.clock - arrival
      self.total_delay += delay
      self.next_departure1 = self.clock + self.service_times_in_queue.pop( - 1)

  def departure2(self):
    if self.num_in_queue == 0:  # if no customer in the queue
      self.server2_status = 0
      self.next_departure2 = float('infinity')

    else:
      self.num_in_queue -= 1
      self.num_of_delays += 1      
      arrival = self.times_of_arrivalqueue.pop(-1)      

      delay = self.clock - arrival
      self.total_delay += delay
      self.next_departure2 = self.clock + self.service_times_in_queue.pop(- 1)


  def update_register(self):
    time_differnce = self.clock - self.last_event_time
    self.area_under_q += self.num_in_queue * time_differnce
    self.area_under_b1 += self.server1_status * time_differnce
    self.area_under_b2 += self.server2_status * time_differnce
    self.last_event_time = self.clock


class SJF:
  def __init__(self):
    self.interarrivals= [0.4,1.2,0.5,1.7,0.2,1.6,0.2,1.4,1.9,.7]  #exponential distribution mean 1/3
    self.service_times= [2.2,0.5,0.3,1.1,3.5,0.6,0.9,1.5,1.2,.9]
    #self.interarrivals= list(np.random.exponential(1/3.0,105))
    #self.service_times=list(np.random.exponential(1/4.0,105))
    print(self.interarrivals)
    print(self.service_times)
    self.clock = 0.0

    self.next_arrival = self.interarrivals.pop(0)
    self.next_departure1 = float('inf')
    self.next_departure2 = float('inf')

    self.num_in_queue = 0
    self.times_of_arrivalqueue = []  # store times of arrivals who are waiting in the queue
    self.service_times_in_queue = []  # store service times of waiting customers in the queue

    self.total_delay = 0.0
    self.num_of_delays = 0.0
    self.area_under_q = 0.0
    self.area_under_b1 = 0.0
    self.area_under_b2 = 0.0
    

    self.server1_status = 0
    self.server2_status = 0  # 0 for IDLE , 1 for BUSY
    self.last_event_time = 0.0  # we will need to store last event clock time

        
  def start(self):
    while self.num_of_delays <=6:
      self.timing()

  def timing(self):   
    self.clock = min(self.next_arrival, self.next_departure1, self.next_departure2)  # First set clock to minimum time of next event
    self.update_register()

    if self.next_arrival < self.next_departure1 and self.next_arrival < self.next_departure2:
      print("Arrival at Clock:" + str(self.clock))
      self.arrival()

    elif self.next_departure1 < self.next_departure2:
      print("Departure of server 1 at " + str(self.clock))
      self.departure1()

    elif self.next_departure2 < self.next_departure1:
      print("Departure of server 2 at " + str(self.clock))
      self.departure2()


    print("Server Status 1:" + str(self.server1_status))
    print("Server Status 2:" + str(self.server2_status))
    print("Times of arrivals in Queue: " + str(self.times_of_arrivalqueue))
    print("Service times in Queue: " + str(self.service_times_in_queue))
    print("Number of Delays: " + str(self.num_of_delays))
    print("Total Delay:" + str(self.total_delay))
    print("Next Arrival Time: " + str(self.next_arrival))
    print("Next Departure Time of s1: " + str(self.next_departure1))
    print("Next Departure Time of s2: " + str(self.next_departure2))
    print("area under q(t):" + str(self.area_under_q))
    print("area under b1(t):" + str(self.area_under_b1))
    print("area under b2(t):" + str(self.area_under_b2))
    print("last event time"+ str(self.last_event_time))
    print("Expected average Delay:" + str(self.total_delay / 6))
    print("UTILIZATION of server1:" + str(self.area_under_b1/self.last_event_time))
    print("UTILIZATION of server2:" + str(self.area_under_b2/self.last_event_time))          
    print("Average number of customers in queue:" + str(self.area_under_q / self.last_event_time))
    print(" ")


  def arrival(self):
    self.next_arrival += self.interarrivals.pop(0)

    if self.server1_status == 0:  # server is idle
      self.server1_status = 1  # make server BUSY
      delay = 0.0  # so delay is zero
      self.total_delay += delay
      self.num_of_delays += 1  # increase the number of customers delayed

      self.next_departure1 = self.clock + self.service_times.pop(0)

    elif self.server2_status == 0:  # server is idle
      self.server2_status = 1  # make server BUSY
      delay = 0.0  # so delay is zero
      self.total_delay += delay
      self.num_of_delays += 1  # increase the number of customers delayed

                # schedule next departure, pop the first element of service_times list to get service time of this customer
      self.next_departure2 = self.clock + self.service_times.pop(0)

    else:  # Server is BUSY
                # increase queue length, this customer will have to wait in the queue
      self.num_in_queue += 1

                # store the arrival time and service time of this customer in seperate lists
      self.times_of_arrivalqueue.append(self.clock)
      self.service_times_in_queue.append(self.service_times.pop(0))
       
       
  def departure1(self):
    if self.num_in_queue == 0:  # if no customer in the queue
      self.server1_status = 0
      self.next_departure1 = float('infinity')

    else:
      self.num_in_queue -= 1
      self.num_of_delays += 1

      index, value = min(enumerate(self.service_times_in_queue), key=operator.itemgetter(1))
      arrival = self.times_of_arrivalqueue.pop(index)##minimun service time arrival will be picked

     

      delay = self.clock - arrival
      self.total_delay += delay
      self.next_departure1 = self.clock + self.service_times_in_queue.pop(index)

  def departure2(self):
    if self.num_in_queue == 0:  # if no customer in the queue
      self.server2_status = 0
      self.next_departure2 = float('infinity')

    else:
      self.num_in_queue -= 1
      self.num_of_delays += 1

      index, value = min(enumerate(self.service_times_in_queue), key=operator.itemgetter(1))
      arrival = self.times_of_arrivalqueue.pop(index)

      
      delay = self.clock - arrival
      self.total_delay += delay
      self.next_departure2 = self.clock + self.service_times_in_queue.pop(index)      
  def update_register(self):
    time_differnce = self.clock - self.last_event_time
    self.area_under_q += self.num_in_queue * time_differnce
    self.area_under_b1 += self.server1_status * time_differnce
    self.area_under_b2 += self.server2_status * time_differnce
    self.last_event_time = self.clock



print("QUEUE USES FIFO")
model = FIFO()
model.start()
print("*****************************************************************************************************")
print("QUEUE USES LIFO")
model2= LIFO()
model2.start()
print("*****************************************************************************************************")
print("QUEUE USES SJF")
model3= SJF()
model3.start()

QUEUE USES FIFO
[0.26529150272103663, 0.41864358765527926, 0.30774104860135626, 0.2624003841057569, 0.18368283036516642, 0.34605309831453646, 0.19183973305621327, 0.7411748043442189, 1.1049707273512712, 0.1612007003007511, 0.5229653799897227, 0.25089136984552945, 0.27981094891294095, 0.8660847179751469, 0.02456178497094929]
[0.02279025727975012, 0.0051063966710562375, 0.44687191990372205, 0.37644605796630387, 0.5100785702363877, 0.9613054595417123, 0.40130988973753645, 0.15473236465324214, 0.3791339942200379, 0.0314686029576357, 0.2553579715980182, 0.0386824206819113, 0.7236051124228815, 0.18445681957776466, 0.13389142957518907]
Arrival at Clock:0.26529150272103663
Server Status 1:1
Server Status 2:0
Times of arrivals in Queue: []
Service times in Queue: []
Number of Delays: 1.0
Total Delay:0.0
Next Arrival Time: 0.683935090376316
Next Departure Time of s1: 0.2880817600007868
Next Departure Time of s2: inf
area under q(t):0.0
area under b1(t):0.0
area under b2(t):0.0
last event time0.2

In [ ]:
import operator
service_times_in_queue=[0.2,0.1,0.3]
index, value = min(enumerate(service_times_in_queue), key=operator.itemgetter(1))
print(index)
print(value)

1
0.1
